In [2]:
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0
    buy_price = 0
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)
get_max_loss(1300, 4.5, 7, 0.2, 0.2)

15111.6056

In [2]:
#타임스켐프 시간 변환
def get_time_hhmmss(mili_time):
    mili_time = float(mili_time)/1000
    KST = datetime.timezone(datetime.timedelta(hours=9))
    dt = datetime.datetime.fromtimestamp(mili_time, tz=KST)
    timeline = str(dt.strftime('%D %H:%M:%S'))
    return timeline

#단계별 구매 수량
def get_open_amt_list(open_amt_unit, open_cnt_limit, increace_rate):
    open_amt = 0
    open_amt_list = [0.0]
    for idx in range(0, open_cnt_limit):
        temp_amt = open_amt_unit + open_amt * increace_rate
        open_amt = round(open_amt + temp_amt, 4)
        open_amt_list.append(open_amt)
    return open_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, open_amt_unit, open_cnt_limit, increace_rate, max_loss_rate):
    open_amt = 0
    open_price = 0
    for idx in range(0, open_cnt_limit):
        temp_amt = open_amt_unit + open_amt * increace_rate
        open_price = round(open_price + close * temp_amt, 4)
        open_amt = round(open_amt + temp_amt, 4)
    return round(open_price * max_loss_rate, 4)


def run_test(config):
    revenue_rate  = config['revenue_rate']  #익절 비율
    max_loss_rate = config['max_loss_rate']   #손절 비율: 20%
    increace_rate = config['increace_rate']
    open_cnt_limit = 7   #최대 오픈 건수
    open_amt_unit = 9     #최소 오픈 수량
    close = 0.6
    open_amt_list = get_open_amt_list(open_amt_unit, open_cnt_limit, increace_rate)
    max_loss = get_max_loss(close, open_amt_unit, open_cnt_limit, increace_rate, max_loss_rate)
#     print("*open_amt_list:{} max_loss:{}".format(open_amt_list, max_loss))

    open_long_cnt = 0 
    open_long_price = 0
    open_long_amt = 0
    open_short_cnt = 0
    open_short_price = 0
    open_short_amt = 0
    revenue = 0
    revenue_t = 0

    df = df_org.iloc[df_org.shape[0]-14400:,]
    # df = df_org
    for i in range(0, df.shape[0]-1):

        close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
        close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4)
        vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
        t = round(df.iloc[i+1:i+2,]['t'].values[0],4)

        #long position stop loss
        long_loss = open_long_price - close2*open_long_amt
        if long_loss > max_loss:
            revenue_t = close2*open_long_amt - open_long_price
            revenue = round(revenue + revenue_t,4)
#             print("*[{}] time:{} long stop loss:{} revenue:{} ".format(i, get_time_hhmmss(t), revenue_t, revenue))        
            open_long_cnt = 0
            open_long_amt = 0
            open_long_price = 0
            continue

        #short position stop loss
        short_loss = close2*open_short_amt - open_short_price
        if short_loss > max_loss:
            revenue_t = open_short_price - close2*open_short_amt
            revenue = round(revenue + revenue_t,4)
#             print("*[{}] time:{} short stop loss:{} revenue:{}".format(i, get_time_hhmmss(t), revenue_t, revenue))         
            open_short_cnt = 0
            open_short_amt = 0        
            open_short_price = 0
            momentum = ''
            continue

        #long take profit
        long_revenue = close2*open_long_amt - \
                            (open_long_price + open_long_price*revenue_rate)
        if open_long_cnt > 0 and long_revenue > 0:
            revenue_t = close2*open_long_amt - open_long_price
            revenue = round(revenue + revenue_t,4)
            open_long_cnt = 0
            open_long_amt = 0
            open_long_price = 0
            continue

        #short take profit
        short_revenue = open_short_price - \
                            (close2*open_short_amt + close2*open_short_amt*revenue_rate)
        if open_short_cnt > 0 and short_revenue > 0:
            revenue_t = open_short_price - close2*open_short_amt
            revenue = round(revenue + revenue_t,4)
            open_short_cnt = 0
            open_short_amt = 0        
            open_short_price = 0
            continue

        #long position open
        if open_long_cnt < open_cnt_limit and close1 < vwap:
            temp_amt = open_amt_unit + open_long_amt*increace_rate
            open_long_price = round(open_long_price + close2*temp_amt,4)
            open_long_amt = round(open_long_amt + temp_amt,4)
            open_long_cnt = open_long_cnt + 1 

        #short position open        
        if open_short_cnt < open_cnt_limit and close1 > vwap:
            temp_amt = open_amt_unit + open_short_amt*increace_rate
            open_short_price = round(open_short_price + close2*temp_amt,4)
            open_short_amt = round(open_short_amt + temp_amt,4)
            open_short_cnt = open_short_cnt + 1  

    #     print("c1:{} c2:{} vwap:{} olc:{} ola:{} olp:{} osc:{} osa:{} osp:{} revenue:{}"
    #           .format(close1, close2, vwap, open_long_cnt, open_long_amt, open_long_price, 
    #                   open_short_cnt, open_short_amt, open_short_price, revenue))

    #print("*revenue:{}".format(revenue))
    return revenue
'''
 revenue_rate = 0.011  #익절 비율
    max_loss_rate = 0.18   #손절 비율: 20%
    increace_rate = 0.25  
'''

'\n revenue_rate = 0.011  #익절 비율\n    max_loss_rate = 0.18   #손절 비율: 20%\n    increace_rate = 0.25  \n'

In [4]:
def black_box_function(revenue_rate, max_loss_rate, increace_rate):
    config_data = {
        'revenue_rate':revenue_rate,
        'max_loss_rate':max_loss_rate,
        'increace_rate':increace_rate
    }
    revenue = run_test(config_data)
    return revenue

pbounds = {
    'revenue_rate':(0.005,0.015),
    'max_loss_rate':(0.1,0.5),
    'increace_rate':(0.15,0.3)
}

optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds=pbounds,
    random_state=1,
)

#실행
optimizer.maximize(
    init_points=5,
    n_iter=50
)

|   iter    |  target   | increa... | max_lo... | revenu... |
-------------------------------------------------------------
|  1        |  9.504    |  0.2126   |  0.3881   |  0.005001 |
|  2        | -9.496    |  0.1953   |  0.1587   |  0.005923 |
|  3        |  7.304    |  0.1779   |  0.2382   |  0.008968 |
|  4        |  15.37    |  0.2308   |  0.2677   |  0.01185  |
|  5        |  6.702    |  0.1807   |  0.4512   |  0.005274 |
|  6        |  16.13    |  0.2084   |  0.3125   |  0.015    |
|  7        |  11.3     |  0.2298   |  0.415    |  0.006749 |
|  8        |  17.96    |  0.3      |  0.3274   |  0.015    |
|  9        |  0.6851   |  0.1841   |  0.1622   |  0.01447  |
|  10       |  13.28    |  0.2589   |  0.3914   |  0.006947 |
|  11       |  11.79    |  0.2205   |  0.2694   |  0.009719 |
|  12       |  12.34    |  0.2863   |  0.3444   |  0.01277  |
|  13       |  15.61    |  0.2233   |  0.4365   |  0.01479  |
|  14       |  15.54    |  0.1984   |  0.4054   |  0.01357  |
|  15   

In [7]:
target_list = []
i=0
for res in optimizer.res:
    target_list.append([res["target"], i])
    i=i+1
target_list.sort(reverse=True)    
target_list

[[23.9152, 16],
 [23.821, 48],
 [22.3041, 17],
 [21.9908, 34],
 [21.6467, 35],
 [21.2034, 18],
 [20.4379, 15],
 [20.1821, 14],
 [20.1602, 53],
 [19.0686, 33],
 [19.0479, 20],
 [18.4825, 47],
 [18.4107, 45],
 [17.9624, 7],
 [17.756, 21],
 [16.5353, 52],
 [16.3376, 19],
 [16.1317, 5],
 [15.6079, 12],
 [15.5384, 27],
 [15.5377, 13],
 [15.365, 3],
 [15.2118, 29],
 [14.8918, 28],
 [14.726, 32],
 [14.7172, 40],
 [14.2423, 41],
 [14.1789, 39],
 [13.9972, 23],
 [13.2808, 9],
 [12.3363, 11],
 [11.7944, 10],
 [11.7688, 46],
 [11.6187, 43],
 [11.2992, 6],
 [9.5043, 0],
 [9.4173, 44],
 [9.3478, 31],
 [8.5843, 30],
 [7.3038, 2],
 [6.7736, 42],
 [6.7021, 4],
 [6.1123, 22],
 [5.229, 51],
 [2.1945, 50],
 [1.5374, 26],
 [0.6851, 8],
 [-1.859, 49],
 [-5.6722, 25],
 [-6.7167, 36],
 [-9.4959, 1],
 [-9.8996, 24],
 [-10.644, 37],
 [-11.7447, 38],
 [-24.6053, 54]]

In [8]:
optimizer.res[16]

{'target': 23.9152,
 'params': {'increace_rate': 0.29730847342510186,
  'max_loss_rate': 0.4079003720626158,
  'revenue_rate': 0.015}}